In [ ]:
import numpy as np
import datetime
import pickle
from collections import Counter

from module_.dataLoader import dataLoader

In [ ]:
# Data Statistics of single-user activity in adlmr

with open("./dataset/preprocessed/adlmr/single/data.pickle", "rb") as f:
    data = pickle.load(f)

resident = "1"

file_sensors = []
file_gaps = []
file_most = []
for i in range(1, 27):
    # Load file > stream > resident
    filename = f'P0{i}' if len(str(i))==1 else f'P{i}'
    single_user_stream = data[filename]['stream'][resident]
    stream = []
    for i in range(len(single_user_stream)):
        label = None
        if single_user_stream[i][3]==resident:
            label = single_user_stream[i][4]
        else:
            label = single_user_stream[i][6]
            assert single_user_stream[i][5]==resident, f"line {i}: Resident Error"
        stream.append(single_user_stream[i][:3]+[label])
    stream = np.array(stream)

    # Divide stream into activity chunks
    transitions = [j for j in range(1, len(stream)) if stream[j,-1]!=stream[j-1, -1]] + [len(stream)]
    prev = 0
    single_activities = []
    for i in range(len(transitions)):
        activity = stream[prev:transitions[i]]
        assert len(set(activity[:,-1]))==1, f"{prev, transitions[i]} No single activity"
        single_activities.append(
            activity
        )
        prev = transitions[i]

    # Average 
    avg_sensors = []
    avg_gaps = []
    avg_most = []
    for sa in single_activities:
        if len(sa)<10 or sa[0, -1] in ['3', '7', '11', '13']:
            continue
        sa_sensors = set(sa[:,0])
        sa_most = Counter(sa[:,0]).most_common(1)[0][1]/len(sa[:,0])
        sa_gaps = []
        for i in range(1, len(sa)):
            sa_gaps.append(float(sa[i,2])-float(sa[i-1,2]))

        sa_gaps = sum(sa_gaps)/len(sa_gaps)

        avg_sensors.append(len(sa_sensors))
        avg_gaps.append(sa_gaps)
        avg_most.append(sa_most)
        # print(len(sa_sensors), sa_gaps)
        # print(sa_counter)

    file_sensors.append(
        sum(avg_sensors)/len(avg_sensors)
    )

    file_gaps.append(
        sum(avg_gaps)/len(avg_gaps)
    )

    file_most.append(
        sum(avg_most)/len(avg_most)
    )

print(sum(file_sensors)/len(file_sensors))
print(sum(file_gaps)/len(file_gaps))
print(sum(file_most)/len(file_most))


In [ ]:
episodes, transitions, labels = dataLoader('testbed')

In [50]:
i = 0
pair, transition, label = episodes[i], transitions[i], labels[i]
print(label)

len(pair[transition:]), len(pair[:transition])
# pair[transition:]
pair[:transition]

Chatting82-Discussion35


array([['Motion8', 'true', '1526127490'],
       ['Motion8', 'true', '1526127520'],
       ['Motion8', 'true', '1526127551'],
       ...,
       ['Motion1', 'true', '1526130114'],
       ['Motion8', 'true', '1526130115'],
       ['Motion2', 'true', '1526130115']], dtype='<U10')

In [ ]:
single_activities = [episodes[i][:transitions[i]] for i in range(len(episodes))]

# Average 
avg_sensors = []
avg_gaps = []
avg_most = []
for sa in single_activities:

    sa_sensors = set(sa[:,0])
    sa_most = Counter(sa[:,0]).most_common(1)[0][1]/len(sa[:,0])
    sa_gaps = []
    for i in range(1, len(sa)):
        sa_gaps.append(float(sa[i,2])-float(sa[i-1,2]))
    try:
        sa_gaps = sum(sa_gaps)/len(sa_gaps)
    except ZeroDivisionError:
        print(sa, transitions)

    avg_sensors.append(len(sa_sensors))
    avg_gaps.append(sa_gaps)
    avg_most.append(sa_most)
    # print(len(sa_sensors), sa_gaps)
    # print(sa_counter)

print(sum(avg_sensors)/len(avg_sensors))
print(sum(avg_most)/len(avg_most))
print(sum(avg_gaps)/len(avg_gaps))


In [ ]:
# Testbed event interval

# index = 4
# episode, transition, label = episodes[index], transitions[index], labels[index]

episode = np.load("./dataset/testbed/npy/seminar/MO/Presentation55.npy")

sensor_list = [item for item in sorted(set(episode[:,0])) if item[0]=="M"]

sensor_active_state = {item: [] for item in sensor_list}
sensor_current_state = {item: False for item in sensor_list}
sensor_active_start = {item: 0. for item in sensor_list}
sensor_active_len = {item: 0. for item in sensor_list}


for i in range(len(episode)):
    s, v, t = episode[i]
    t = float(t)-float(episode[0,2])

    if s[0]!="M": continue
    if v=="true":
        if sensor_current_state[s]==False: # New active state
            sensor_active_start[s] = t
            sensor_active_len[s] = 1
            sensor_current_state[s] = True
        else:
            sensor_active_len[s]+=1
    elif v=="false":
        if sensor_current_state[s]==True:
            sensor_active_state[s].append(
                (sensor_active_start[s], t, sensor_active_len[s])
            )
            sensor_current_state[s] = False
            sensor_active_len[s] = 0
        else:
            # print(i)
            pass
    
    if i==len(episode)-1:
        for sensor, state in sensor_current_state.items():
            if state==True:
                if sensor_active_start[sensor]-t!=0:
                    sensor_active_state[sensor].append(
                        (sensor_active_start[sensor], t, sensor_active_len[sensor])
                    )

for sensor, states in sensor_active_state.items():
    print(sensor)
    length_set = []
    for state in states:
        begin_, end_, true_num = state
        # length_set.append((end_-begin_, true_num))
        length_set.append(int((end_-begin_)/true_num))
    print(Counter(length_set) )

print(datetime.datetime.fromtimestamp(float(episode[0,2])), len(episode))